# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [17]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [18]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [19]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
total=data.groupby(['CustomerID','ProductName'],as_index=False)['Quantity'].sum()
total.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot=pd.pivot_table(total,index='ProductName',columns='CustomerID',values='Quantity').fillna(0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist,squareform

In [7]:
#squareform transforms
similarity=pd.DataFrame(1/(1+squareform(pdist(pivot.T,'euclidean'))),
                        index=pivot.columns,columns=pivot.columns)
similarity.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
similar_412=similarity.iloc[4,:].sort_values(ascending=False)[1:6]
similar_412

CustomerID
3317    0.085638
3535    0.084959
1008    0.083333
1072    0.082709
639     0.082403
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
total_sim_412=total[total.CustomerID.isin(similar_412.index)]
total_sim_412.head()

,CustomerID,ProductName,Quantity
452,639,Anchovy Paste - 56 G Tube,1
453,639,Appetizer - Mushroom Tart,1
454,639,Assorted Desserts,1
455,639,Bread - Italian Roll With Herbs,1
456,639,Bread - Raisin Walnut Oval,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
products_412=total_sim_412.groupby('ProductName',as_index=False).Quantity.sum().sort_values('Quantity',ascending=False)
products_412.head()

,ProductName,Quantity
123,Olive - Spread Tapenade,3
175,Sprouts - Baby Pea Tendrils,3
201,Wine - Blue Nun Qualitatswein,3
193,"Veal - Inside, Choice",3
134,"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
merge=pivot.merge(products_412,on='ProductName')
merge[merge[412]==0].sort_values('Quantity',ascending=False).head()

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
153,"Sauce - Gravy, Au Jus, Mix",0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
175,Sprouts - Baby Pea Tendrils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,3
123,Olive - Spread Tapenade,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
201,Wine - Blue Nun Qualitatswein,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,3
37,Cheese - Cambozola,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,3


In [12]:
values=merge.ProductName[:5].values
values

array(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
       'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
       'Apricots - Dried'], dtype=object)

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
recomendations={}
customers=data.CustomerID.unique()

In [14]:
for customer in customers:
    sim_customer=similarity.loc[customer,:].sort_values(ascending=False)[1:6]
    total_sim_customer=total[total.CustomerID.isin(sim_customer.index)]
    products_customer=total_sim_customer.groupby('ProductName',as_index=False).Quantity.sum().sort_values('Quantity',ascending=False)
    merge_customer=pivot.merge(products_customer,on='ProductName')
    values_customer=merge_customer.ProductName[:5].values
    recomendations[customer]=values_customer

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [15]:
results=pd.DataFrame.from_dict(recomendations,orient='index').reset_index()
columns=['CustomerID','Recomendation1','Recomendation2','Recomendation3','Recomendation4','Recomendation5']
results.columns=columns
results.head()

,CustomerID,Recomendation1,Recomendation2,Recomendation3,Recomendation4,Recomendation5
0,61288,Appetizer - Mushroom Tart,Apricots - Halves,Arizona - Green Tea,Assorted Desserts,Bacardi Breezer - Tropical
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Arizona - Green Tea,Assorted Desserts,Bacardi Breezer - Tropical
2,40094,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem
3,23548,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
4,78981,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Halves,Apricots Fresh,Artichokes - Jerusalem


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [16]:
similarity=pd.DataFrame(1/(1+squareform(pdist(pivot.T,'cityblock'))),
                        index=pivot.columns,columns=pivot.columns)
recomendations={}
customers=data.CustomerID.unique()
for customer in customers:
    sim_customer=similarity.loc[customer,:].sort_values(ascending=False)[1:6]
    total_sim_customer=total[total.CustomerID.isin(sim_customer.index)]
    products_customer=total_sim_customer.groupby('ProductName',as_index=False).Quantity.sum().sort_values('Quantity',ascending=False)
    merge_customer=pivot.merge(products_customer,on='ProductName')
    values_customer=merge_customer.ProductName[:5].values
    recomendations[customer]=values_customer
results=pd.DataFrame.from_dict(recomendations,orient='index').reset_index()
columns=['CustomerID','Recomendation1','Recomendation2','Recomendation3','Recomendation4','Recomendation5']
results.columns=columns
results.head()

,CustomerID,Recomendation1,Recomendation2,Recomendation3,Recomendation4,Recomendation5
0,61288,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Bandage - Fexible 1x3,Bandage - Flexible Neon
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
2,40094,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
3,23548,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves,Apricots Fresh
4,78981,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Assorted Desserts
